###### This code produces SARIMAX models for each stock item in the test set, makes predictions for a validation period and calculates predictions errors

In [1]:
# import libararies
# importamos las librerias necesarias
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

pd.options.mode.chained_assignment = 'raise'

In [2]:
# read data sets
# importamos los datos  
data_train = pd.read_csv('Modelar_UH2021.txt', sep='|', low_memory=False)
data_test = pd.read_csv('Estimar_UH2021.txt', sep='|', low_memory=False)

# select columns needed for further time series modelling
# seleccionamos los campos que vamos a necesitar para el modelado de series temporales
data = data_train.loc[:,['fecha','id','dia_atipico','campaña','unidades_vendidas']]


# eliminate duplicates
# eliminamos duplicados
data = data.drop_duplicates()

# change dates data type to 'timestamps' 
# cambiamos formato de las fechas a "timestamp"
data['fecha']=pd.to_datetime(data['fecha'], format="%d/%m/%Y %H:%M:%S")

In [3]:
# unique stock codes in the test dataset
# los articulos en el conjunto de test
codes_test = data_test['id'].unique()

In [ ]:
# create and save separate files for each stock code to be predicted
# creamos y guardamos los archivos separados para los articulos que debemos pronosticar
for each in codes_test:
    df = data[data['id']==each].copy()
    path = './files/' +str(each)+'.csv'
    df.to_csv(path, index=False)

In [4]:
# function for data preprocessing
def prep_data (stock_code):
    
    # read previously saved file for an individual stock code
    path = './files/' +str(each)+'.csv'   
    df = pd.read_csv(path)
    
    # create dataframe for the stock code's time series
    df_ = pd.DataFrame(index=pd.date_range('2015-06-01', periods=488, freq='D'))

    # mark duplcates to eliminate
    df.loc[:,'delete']=0
        
    fechas_dup = pd.Series(df['fecha'])[pd.Series(df['fecha']).duplicated()].values
    for fecha in fechas_dup:
        index = df[(df['fecha'] ==fecha ) & (df['campaña']==0)].index
        df.loc[index, 'delete']=1
    
    # eliminate duplicates
    index_names = df[df['delete'] == 1 ].index 
    df.drop(index_names, inplace = True)
    
    # change the index
    df.index=df.loc[:,'fecha']
    df.index = pd.to_datetime(df.index)

    # for each day of promotion ('campaña'=1), change 'dia_atipico' to zero to have these codes independent 
    for i in range(len(df)):
        if (df.loc[df.index[i],'campaña']==1):
            df.loc[df.index[i],['dia_atipico']]=0   

    # eliminate the columns which are not longer required
    df = pd.DataFrame(df[['unidades_vendidas', 'visitas','campaña','dia_atipico']])

    # add time series to its dataframe, fill in the gaps with zeroes
    ts = pd.concat([df_,df], axis=1)
    ts = ts.fillna(value=0)
    
    # limit the series to the last year of observations
    ts=ts.iloc[-365:,:]

    return(ts)

In [53]:
# function for modeling with SARIMAX
def modeling (ts):

    # divide data in train and validation sets
    train = ts.iloc[:-90,:]
    test = ts.iloc[-90:,:]

    # look for best model for the series with 'autoarima'
    res = auto_arima(ts['unidades_vendidas'], exogenous=ts[['campaña', 'dia_atipico']], m=7,
                     suppress_warnings=True, enforce_invertibility=False)
    
    order = res.order
    seasonal_order = res.seasonal_order
    
    
    # fit the model using train set
    model = SARIMAX(train['unidades_vendidas'], exog=train[['visitas','campaña', 'dia_atipico']],
                    order=order,
                    seasonal_order = seasonal_order).fit()
    
    # make predictions for validation set
    start=len(train)
    end=len(train)+len(test)-1
    predictions = model.predict(start=start, end=end, exog=test[['visitas','campaña', 'dia_atipico']])
    
    
    # change negative values to zero
    for i in range(len(predictions)):
        if predictions[i] < 0:
            predictions[i] = 0
    
    # calculate prediction error
    mae = mean_absolute_error(test['unidades_vendidas'], predictions)
    
    # calculate proportion MAE/STD
    coef = mae / (test['unidades_vendidas'].std()+0.001)
    
    return ([order, seasonal_order, mae, coef])

In [6]:
# Set a dictionary for keeping the modeling results
# El diccionario para guardar los resultados del modelado
results_sarima = {}

In [ ]:
### DONT FORGET TO CHANGE!! missing_codes -> codes_test

In [8]:
### DONT FORGET TO FELETE
parameters =  pd.read_csv('sarima_models.csv')
codes = parameters['id'].to_list()
missing_codes = []
for each in codes_test:
    if each not in codes:
        missing_codes.append(each)
len(missing_codes)

898

In [55]:
# complete dictionary with found models' parameters for all stock codes
# TIME CONSUMING!
for each in missing_codes[847:]:#codes_test:
    
    ts = prep_data(each)
    results_sarima[each] = modeling(ts)
    mae = results_sarima[each][2]
    coef =results_sarima[each][3]
    
    # print the progress line
    print('Item number: '+str(len(results_sarima)) + ', Item code: '+ str(each)+ ', MAE: '+str(mae)+', Coef: '+ str(coef))

C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:994: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1006: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Item number: 842, Item code: 431976, MAE: 6.094059917447479, Coef: 0.712876134999981


C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:994: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1006: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Item number: 843, Item code: 432100, MAE: 78.59300904036493, Coef: 3.7687958334233476


C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:994: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Item number: 844, Item code: 432116, MAE: 6.922006288386363, Coef: 0.7063030288041544


C:\Users\Tanya\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


LinAlgError: Non-positive-definite forecast error covariance matrix encountered at period 1

In [54]:
missing_codes[746]
missing_codes[782]
missing_codes[827]
missing_codes[834]
missing_codes[845]
missing_codes[846]

431972

In [ ]:
####

In [58]:
# save results to a dataframe
# guardamos los resultados en dataframe
sarima_res=pd.DataFrame(columns=['id','order','seasonal_order','mae','coef'])

id=[]
order=[]
seasonal_order=[]
mae=[]
coef=[]

for each in results_sarima:
    id.append(each)
    order.append(results_sarima[each][0])
    seasonal_order.append(results_sarima[each][1])
    mae.append(results_sarima[each][2])
    coef.append(results_sarima[each][3])
    
sarima_res['id']=id
sarima_res['order']=order
sarima_res['seasonal_order']=seasonal_order
sarima_res['mae']=mae
sarima_res['coef']=coef

# save the dataframe as a file
# guardamos dataframe como archivo
sarima_res.to_csv('sarima_res_.csv', index=False)

In [59]:
len(sarima_res)

844